# Intro to Mininet with Remote Ryu Controller 
    
In this tutorial, you will experiment with the Ryu & Mininet.

All the code for this tutorial can be found here
[Mininet and Ryu](https://github.com/dabun01/Mininet_Ryu_Tutorial)
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

<br><img src="./images/Highleveltopology.png" alt="topology"><br>

<br><br>This is the second step in this assignment. To go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [1]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = "MininetWorkspace"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,a70de2f5-9e12-4b6b-b412-0ae1a2c553b0
Bastion Host,bastion.fabric-testbed.net
Bastion Username,dabun01_0000229645
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e620bad4-a51e-4308-9167-029432d28338,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w5.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe94:a0a3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe94:a0a3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
87de401c-b4c3-471f-997b-ee5d8c9fee14,Host,1,2,10,default_ubuntu_20,qcow2,eduky-w10.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe43:f102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe43:f102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e620bad4-a51e-4308-9167-029432d28338,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w5.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe94:a0a3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe94:a0a3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
87de401c-b4c3-471f-997b-ee5d8c9fee14,Host,1,2,10,default_ubuntu_20,qcow2,eduky-w10.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe43:f102,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe43:f102,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## 2. Guided Experiment

### 2.1. Login to your hosts and preconfigurement  ¶
1.	Locate the SSH Commands:
In the Node Table, find the SSH Command section. These commands are used to access the nodes via the terminal.
2.	Access the Host Node:
    - Identify the SSH command for the Host Node.
    - Copy the SSH command.
	- Open the terminal by clicking the blue plus (+) icon to launch a new window, then select Terminal under the Other section.
	- Paste the SSH command into the terminal. Once executed, you should be inside the Host Node.
3.	Access the Controller Node:
	-	Repeat the same process for the Controller Node.
	-	Locate and copy its corresponding SSH command.
	-	Paste the command into another terminal window to access the Controller Node.

Preparing for the Tutorial

4. Ensure you have the following terminal windows open:
	- One window with SSH access to the Controller Node.
	- One window with SSH access to Host1.
### 2.1. SDN Introduction
What is SDN
Software defined networking (SDN) is an approach to network management that enables dynamic programmable network configuration to improve network performance and monitoring. It is a new way to manage computer network that makes them easier and more flexible

In a traditional network the hardware like the router and switches decide how to move data through the network, but SDN takes over the decision making to a central software system. This is accomplished by separating the control plane from the data plane. 

The data plane consists of all the activity sent by the end user and includes forwarding packets, segmentation and reassembly of data. The control plane is all the activities that do not involve the end user. The control plane is responsible for creating routing tables and setting packet handling policies. Think of this as the brain of the network. 

In SDNs the software that controls the network is separated from the hardware. The control plane is now managed by a centralized controller while the data plane stays in the hardware. This configuration allows us to control and manage the network without having to configure each individual device instead we can adjust the network from one central place. 

### 2.2 Intro into Mininet

Mininet is a lightweight network emulator that creates a virtual network on a single computer. It is commonly used in Software-Defined Networking (SDN) research, development, and education. Mininet enables users to emulate networks with multiple hosts, switches, and links, allowing them to test and prototype network topologies, applications, and protocols without requiring physical hardware.

#### How to start Mininet
1. Go to the host node as it has Mininet preinstalled
2. Types ` sudo mn` into the command line this will start the default topology 
3. Done you have started Mininet and are now in the Mininet Command Line Interface(CLI)

#### Interact with Host and Switches
Try the command `pingall` to see if the network is functional 

The host should ping each other and you can see that in the terminal

We are given a multitude of commands to interact with the environment. Type `help` to see them all 
`nodes`, `net`, and `dump` are a few commands type each to see what they do.
 
We can also interact with each node and switch individually. If you start your command with the host or the switch name it will be like you are typing into the terminal of each

Try this command `h1 ifconfig -a` 

You should be able to see host `h1-eth0` and the loopback

Type `exit` into the CLI

Now type `sudo mn -c` It is good practice to type this after you are done using Mininet as this will clean up the environment 

We can also use commands to change the topology. Let me show you a few examples.

Type this command into the terminal of the host node. We can use the topo command to change the topology.
`sudo mn --test pingall --topo single,3`

Run the `pingall` command again


### 2.3 Ryu 

Now I will introduce Ryu. In the above examples, we have been using the default controller that comes with Mininet but we want to be able to use our own controller that is where Ryu comes into play. Ryu is an open source Python based controller perfect for testing and learning about SDNs. 

1. Login to the Controller 
2. Start your Ryu controller with the following command :

``ryu-manager ryu.app.simple_switch``

Then go back to the Host 1 and in the command line for the host not the Mininet CLI type the following command

`sudo mn --controller=remote,ip=10.0.0.2,port=6633 --topo single,2 --mac`

Now use the `pingall` command to test the connection and on the controller we will be able to see the packets being sent.

Type `h1 ping h2 -c 6` and take a note of time of the first packet compared to the rest

The first packet takes longer to send because it has to go to the controller first then to its final destination 

We also have access to the terminal of the switch. Type this command to see basic info for the switch

`s1 sudo ovs-vsctl show`

You will be able to see to the controller the switch is connected to and the IP address of our external controller. In addition to fail mode and ports.

#### 2.3.1 standalone vs secure mode
The OpenFlow controller manages all flow setups on the switch, meaning that when the controller is not running, no packet switching should occur. However, depending on your network’s configuration, this behavior might not always be ideal. For some setups, it might be preferable for the switch to default to a learning Layer 2 switch when the controller is unavailable. In other cases, this fallback could be undesirable.

Open vSwitch (OVS) allows you to adjust this behavior through a tunable parameter called fail-safe-mode, which can be configured as follows:
-   **standalone** (default): OVS takes over packet forwarding responsibilities if the controller is unavailable.
-   **secure**: The controller is solely responsible for packet forwarding. If the controller is down, all packets are dropped.

If the fail-safe mode is not explicitly set, OVS defaults to standalone mode. For this tutorial, we will set the fail-safe mode to secure, as we want precise control over packet forwarding.

This command provides flow information for all switches.
`dpctl dump-flows`

And now you have started the controller and connected it to a simple network topology. In the next section we will introduce how to write your own network topologies using python scripts and how to automatically start your controller and build a topology with just one command


### 2.4 Mininet Custom Topology
We can also write Mininet custom topologies. Let’s start with a basic 2 host 2 switch topology. In the GitHub repository, I have included a skeleton framework to get you started along with the answers. Go to the **myFirstTopo.py** 

This is the topology we want to create 
`host --- switch --- switch --- host` 

 - Minnet gives us methods to easily add switches and hosts and then connect them

 Key functions: `addHost(“name”)` , `addSwitch(“name”)`, ``addLink(varibleName,varibleName)``

 For the name, you want to use the Mininet naming convention of **h1, h2, h3,** etc for hosts and **s1, s2, s3,** etc for switches. 

 Start by creating the host on the left side. Here is an example code to help you get started. **leftHost** is our variable name    that can be anything you want.

`leftHost = self.addHost(“h1”)`

 Now write the code to add the left switch. It will look similar to the line above. Add the right switch and host

 And finally, link them together in a straight line to create our simple topology.

`self.addLink(varibleName,varibleName)`

 Congrats you have made your first topology. Now we can go test the topology.

Go to the host node and try the following command which will start our network

`sudo mn --custom myFirstTopo.py --topo mytopo`

Now we want to try to connect it with our remote controller. Go to the controller and start it.

``ryu-manager ryu.app.simple_switch``

Back on the host node we want to initialize our topology and connect it to our external controller 

`sudo mn --custom myFirstTopo.py --topo mytopo --controller=remote,ip=10.0.0.2,port=6633`


### 2.5 Mesh Topology (Advance)

Before we created a simple linear topology. Now we want to create a mesh topology where every switch is connected to every other switch. There are multiple ways to accomplish this topology. Use this as a reference. I find it easier to draw out the topology first and then write the code. Try to create this topology. My implementation is included in the **mesh.py** file. Use the **myMesh.py** to write the code. 

![Mesh Toplogy Digram](./images/meshTopo.png)

Below I will walk through my implementation. So stop here if you wish to try it on your own

- First, add all the switches and hosts using the addSwitch and addHost methods like before. 
- **Note avoid the use of  `IP = 10.0.0.2` to not conflict with the external controller.** 
- In my case I used ip = `10.0.0.3/24` for host 1 and `10.0.0.4/24` for host 2.

This is how to set the IP address
`H1 = self.addHost('h1', ip='10.0.0.3/24')`

Create a list to hold all my switches
`SwitchList = [S1, S2, S3, S4]`

Then write a for loop function to connect the switches and hosts

**See mesh.py for full code**

### 2.6 Mesh Testing

Now that we have created a mesh topology we want to run it and see it in action. A benefit of a mesh topology is self healing. If a node switch happens for some reason to fail the traffic can be rerouted and the network is still functional 

First we need to start our controller use this command to start the controller (Note we are using a different controller than before)
`ryu-manager ryu.app.simple_switch_stp_13`

We are using a spanning tree protocol controller. The Spanning Tree Protocol (STP) is a network protocol designed to prevent loops in a Layer 2 network, such as a mesh network. It works by organizing switches into a loop-free topology while still allowing redundancy for fault tolerance.

Run the following command to start the topology. (Note you might have to cancel then re-run to get it working)

`sudo mn --custom mesh.py --topo mytopo --controller=remote,ip=10.0.0.2,port=6633`

Now use the `dump` and `net` commands to see the connections

Go to the controller and you can see the STP configuring the role of each switch

-   Root Port: The port with the best path to the root bridge. Each switch (except the root bridge) has one root port.

-   Designated Port: Ports selected to forward traffic to downstream devices. These ports are in the **FORWARD** state.

-   Non-Designated Port: Ports that would form loops if they forwarded traffic. These ports are in the **BLOCK** state.

Wait a minute or two for the controller to do it's job then run `pingall`

Use the `iperf` command to test the bandwidth between the 2 hosts

Like I had mentioned above we can stop a switch and still be able to ping the hosts. (Note in this mesh topology if s1 and s3 are stopped we can not ping the hosts)

-   To stop a switch run this command `switch s2 stop` and to start `switch s2 start`

Stop switch 2 and see what the controller does to account for this change.

-   Try stopping different switches and running `pingall` and `iperf`



### Conclusion

Thank you for using this tutorial I hope it was helpful and you learned a lot about SDN. Feel free to experiment with the different topologies and controllers. 

#### Additional Info 

There is one more topology I would like to highlight. In the repo you will find a a file name torus.py and it creates a very interesting topology

![Torus Toplogy Digram](./images/torus.png)




## Cleanup Resources
Once you have completed the assignment shut down the slice.

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")